# Project2 for COEN272 
# Dengwei Wang 00001599188

In [1]:
#loading training data and test data into array
import numpy
data = numpy.loadtxt("train.txt", dtype = int)
test5 = numpy.loadtxt("test5.txt", dtype = int)
test10 = numpy.loadtxt("test10.txt", dtype = int)
test20 = numpy.loadtxt("test20.txt", dtype = int)
data_arr = [[0 for _ in range(1000)] for _ in range(200)]
test5_arr = [[0 for _ in range(1000)] for _ in range(100)]
test10_arr = [[0 for _ in range(1000)] for _ in range(100)]
test20_arr = [[0 for _ in range(1000)] for _ in range(100)]
for u,m,r in data:
    data_arr[u-1][m-1] = r
for u,m,r in test5:
    test5_arr[u-201][m-1] = r 
for u,m,r in test10:
    test10_arr[u-301][m-1] = r 
for u,m,r in test20:
    test20_arr[u-401][m-1] = r 

# Cosine-Similarity

In [2]:
# function for calculating cosine similarity
def cosine_sim(vector1, vector2):
    if len(vector1) != len(vector2): 
        print("dimension error!!")
        return 0
    v1, v2 = [], []
    num, norm1, norm2 = 0, 0.0, 0.0
    for i,j in zip(vector1,vector2):
        if i != 0 and j != 0:
            v1.append(i)
            v2.append(j)
            num += i*j
            norm1 += i*i
            norm2 += j*j
    if num == 0: return 0
    if len(v1) == 1:  #only one common rating
        return 1 - abs(v1[0] - v2[0])/4  #rate the similarity by 5 ranks based on the difference: 0,0.25,0.5,0.75,1
    return num / ((norm1**0.5)*(norm2**0.5))

In [3]:
# function for calculating cosine similarity ranking list between each user in test data with each user in training data
def get_cos_similarity_mat(training_data, test_data):
    mat = [[0 for _ in range(200)] for _ in range(100)]
    for i in range(100):
        for j in range(200):
            mat[i][j] = cosine_sim(test_data[i], training_data[j])
    return mat

# get the ranking of users based on cosine similarity in descending order
def rank_similarity(mat):
    res, abs_cos_sim = [],[]
    for u in mat:
        for cos in u:
            abs_cos_sim.append(abs(cos))
        res.append(list(numpy.argsort(abs_cos_sim))[::-1])
        abs_cos_sim = []
    return res

In [4]:
cos_sim5 = get_cos_similarity_mat(data_arr, test5_arr)
cos_sim10 = get_cos_similarity_mat(data_arr, test10_arr)
cos_sim20 = get_cos_similarity_mat(data_arr, test20_arr)
rank5 = rank_similarity(cos_sim5)
rank10 = rank_similarity(cos_sim10)
rank20 = rank_similarity(cos_sim20)

In [5]:
# calculating the ranking based on baisc user-based collaborative filtering algorithms -- cosine similarity method
def cos_sim_rating_prediction(test_data, test_ranking, test_cossim_mat, training_data, id_shift, k):
    res_arr = []
    for i in test_data:
        if list(i)[2] == 0:
            res_arr.append(list(i))
    for i,j in enumerate(res_arr):
        user_id = j[0] - id_shift
        movie_id = j[1] - 1
        cnt = 0
        num, deno = 0.0, 0.0
        for m in test_ranking[user_id]:
            cos_sim = test_cossim_mat[user_id][m]
            rate_in_training = training_data[m][movie_id]
            if rate_in_training and cos_sim > 0.3:
                num += cos_sim*rate_in_training
                deno += cos_sim
                cnt += 1
                if cnt >= k: break
        if num == 0 and deno == 0: # rate 3 for default when there are no similar user
            res_arr[i][2] = 3
            continue
        res_arr[i][2] = int(round(num/deno, 0)) #num/deno
    return res_arr
        

In [6]:
rate_res5 = cos_sim_rating_prediction(test5, rank5, cos_sim5, data_arr, 201, 30)
rate_res10 = cos_sim_rating_prediction(test10, rank10, cos_sim10, data_arr, 301, 30)
rate_res20 = cos_sim_rating_prediction(test20, rank20, cos_sim20, data_arr, 401, 30)

In [7]:
def write_file(res, res_file):
    with open(res_file, 'w') as f:
        for l in res:
            f.write(str(l)[1:-1].replace(',',''))
            f.write('\n')

In [8]:
write_file(rate_res5, 'cos_sim_rating_res5.txt')
write_file(rate_res10, 'cos_sim_rating_res10.txt')
write_file(rate_res20, 'cos_sim_rating_res20.txt')

# Rating Prediction -- Pearson Correlation

In [39]:
def calculate_pearson_weight(vec1, vec2): #vectors of same size stand for same set of movies
    v1, v2 = [], []
    for i in range(len(vec1)):
        if vec1[i] != 0 and vec2[i] != 0:
            v1.append(vec1[i])
            v2.append(vec2[i])
    if len(v1) == 0: 
        return 0
    if len(v1) == 1:
        # return 1 - abs(v1[0]-v2[0])/2  #weight  -1, -0.5, 0, 0.5, 1 when only one similar item
        return 1 - abs(v1[0]-v2[0])/4  #weight  0, 0.25, 0.5, 0.75, 1 when only one similar item
    r_a, r_u = sum(v1)/len(v1), sum(v2)/len(v2)
    num, deno = 0.0, 0.0
    for i in range(len(v1)):
        v1[i] -= r_a 
        v2[i] -= r_u
    num = sum([m*n for m,n in zip(v1,v2)])
    deno = (sum([m*m for m in v1])**0.5) * (sum([n*n for n in v2])**0.5)
    if deno == 0: 
        # return 1 - abs(r_a - r_u)/2
        return 1 - abs(r_a - r_u)/4
    weight = num/deno
    return weight

def get_r_u_value(vec1, vec2):
    v1, v2 = [], []
    for i in range(len(vec1)):
        if vec1[i] != 0 and vec2[i] != 0:
            v1.append(vec1[i])
            v2.append(vec2[i])
    if len(v1) == 0: 
        return 0
    return sum(v2)/len(v2)

In [10]:
def get_pearson_weight_mat(training_data, test_data):
    mat = [[0 for _ in range(200)] for _ in range(100)]
    for i in range(100):
        for j in range(200):
            mat[i][j] = calculate_pearson_weight(test_data[i], training_data[j])
    return mat

def get_pearson_r_a_mat(test_data):
    res = []
    for i in test_data:
        cnt, s = 0, 0.0
        for j in i:
            if j > 0:
                cnt += 1
                s += j
        if cnt != 0:
            res.append(s/cnt)
        else: 
            res.append(0)
    return res

def get_pearson_r_u_mat(training_data, test_data):
    r_u = [[0 for _ in range(200)] for _ in range(100)]
    for i in range(100):
        for j in range(200):
            r_u[i][j] = get_r_u_value(test_data[i], training_data[j])
    return r_u

def rank_pearson_weight(mat):
    res, abs_pear_weight = [],[]
    for u in mat:
        for pear in u:
            abs_pear_weight.append(pear) # abs(pear)
        res.append(list(numpy.argsort(abs_pear_weight))[::-1])
        abs_pear_weight = []
    return res

In [11]:
pearson_r_a_test5 = get_pearson_r_a_mat(test5_arr)
pearson_r_a_test10 = get_pearson_r_a_mat(test10_arr)
pearson_r_a_test20 = get_pearson_r_a_mat(test20_arr)
pearson_weight_test5 = get_pearson_weight_mat(data_arr, test5_arr)
pearson_ranking_test5 = rank_pearson_weight(pearson_weight_test5)
pearson_weight_test10 = get_pearson_weight_mat(data_arr, test10_arr)
pearson_ranking_test10 = rank_pearson_weight(pearson_weight_test10)
pearson_weight_test20 = get_pearson_weight_mat(data_arr, test20_arr)
pearson_ranking_test20 = rank_pearson_weight(pearson_weight_test20)

In [12]:
pearson_r_u_test5 = get_pearson_r_u_mat(data_arr, test5_arr)
pearson_r_u_test10 = get_pearson_r_u_mat(data_arr, test10_arr)
pearson_r_u_test20 = get_pearson_r_u_mat(data_arr, test20_arr)

In [38]:
def pearson_rating_prediction(test_data, test_ranking, test_weight_mat, r_a_mat, r_u_mat, training_data, id_shift, k):
    res_arr = []
    for i in test_data:
        if list(i)[2] == 0:
            res_arr.append(list(i))
    for i,j in enumerate(res_arr):
        user_id = j[0] - id_shift
        movie_id = j[1] - 1
        r_a = r_a_mat[user_id]
        cnt = 0
        num, deno = 0.0, 0.0
        for m in test_ranking[user_id]:
            weight = test_weight_mat[user_id][m]
            rate_in_training = training_data[m][movie_id]
            r_u = r_u_mat[user_id][m]
            if rate_in_training and weight > 0.3:
                num += weight*(rate_in_training - r_u)
                deno += abs(weight)
                cnt += 1
                if cnt >= k: break
        if num == 0 and deno == 0: # rate its mean rating for default when there are no similar user
            res_arr[i][2] = int(round(r_a, 0))
            continue
        ans = int(round(r_a + num/deno, 0)) # r_a + num/deno
        if ans > 5: ans = 5
        elif ans < 1: ans = 1
        res_arr[i][2] = ans
    return res_arr

In [36]:
pearson_out5 = pearson_rating_prediction(test5, pearson_ranking_test5, pearson_weight_test5, pearson_r_a_test5, pearson_r_u_test5, data_arr, 201, 30)
pearson_out10 = pearson_rating_prediction(test10, pearson_ranking_test10, pearson_weight_test10, pearson_r_a_test10, pearson_r_u_test10, data_arr, 301, 30)
pearson_out20 = pearson_rating_prediction(test20, pearson_ranking_test20, pearson_weight_test20, pearson_r_a_test20, pearson_r_u_test20, data_arr, 401, 30)

In [37]:
write_file(pearson_out5, 'pearson_rating_res5_05271530.txt')
write_file(pearson_out10, 'pearson_rating_res10_05271530.txt')
write_file(pearson_out20, 'pearson_rating_res20_05271530.txt')

# Pearson Correlation with IUF (Inverse user frequency)

In [17]:
import math
def get_iuf_arr(training_data):
    movie_rating_cnt = [0 for _ in range(1000)]
    iuf_arr = []
    for u_id,movies in enumerate(training_data):
        cnt = 0
        for m_id,rate in enumerate(movies):
            if rate > 0:
                movie_rating_cnt[m_id] += 1
    for n in movie_rating_cnt:
        if n != 0:
            iuf_arr.append(math.log10(200/n)) 
        else:
            iuf_arr.append(1)
    return iuf_arr

def get_iuf_train_arr(training_data, iuf_arr):
    iuf_data_arr = [[0 for _ in range(1000)] for _ in range(200)]
    for i in range(len(training_data)):
        for j in range(1000):
            iuf_data_arr[i][j] = round(training_data[i][j]*iuf_arr[j],3)
    return iuf_data_arr

def get_iuf_test_arr(test_data, iuf_arr):
    iuf_test_arr = [[0 for _ in range(1000)] for _ in range(100)]
    for i in range(len(test_data)):
        for j in range(1000):
            iuf_test_arr[i][j] = round(test_data[i][j]*iuf_arr[j],3)
    return iuf_test_arr

In [18]:
iuf_arr = get_iuf_arr(data_arr)
iuf_data_arr = get_iuf_train_arr(data_arr, iuf_arr)
iuf_test5_arr = get_iuf_test_arr(test5_arr, iuf_arr)
iuf_test10_arr = get_iuf_test_arr(test10_arr, iuf_arr)
iuf_test20_arr = get_iuf_test_arr(test20_arr, iuf_arr)

In [19]:
# recalculate the rating and weight matrix using the modified data
pearson_iuf_r_a_test5 = get_pearson_r_a_mat(iuf_test5_arr)
pearson_iuf_r_a_test10 = get_pearson_r_a_mat(iuf_test10_arr)
pearson_iuf_r_a_test20 = get_pearson_r_a_mat(iuf_test20_arr)
pearson_weight_iuf_test5 = get_pearson_weight_mat(iuf_data_arr, test5_arr)
pearson_ranking_iuf_test5 = rank_pearson_weight(pearson_weight_iuf_test5)
pearson_weight_iuf_test10 = get_pearson_weight_mat(iuf_data_arr, test10_arr)
pearson_ranking_iuf_test10 = rank_pearson_weight(pearson_weight_iuf_test10)
pearson_weight_iuf_test20 = get_pearson_weight_mat(iuf_data_arr, test20_arr)
pearson_ranking_iuf_test20 = rank_pearson_weight(pearson_weight_iuf_test20)

In [20]:
pearson_iuf_r_u_test5 = get_pearson_r_u_mat(iuf_data_arr, iuf_test5_arr)
pearson_iuf_r_u_test10 = get_pearson_r_u_mat(iuf_data_arr, iuf_test10_arr)
pearson_iuf_r_u_test20 = get_pearson_r_u_mat(iuf_data_arr, iuf_test20_arr)

In [21]:
pearson_iuf_out5 = pearson_rating_prediction(test5, pearson_ranking_iuf_test5, pearson_weight_iuf_test5, pearson_r_a_test5, pearson_r_u_test5, data_arr, 201, 30)
pearson_iuf_out10 = pearson_rating_prediction(test10, pearson_ranking_iuf_test10, pearson_weight_iuf_test10, pearson_r_a_test10, pearson_r_u_test10, data_arr, 301, 30)
pearson_iuf_out20 = pearson_rating_prediction(test20, pearson_ranking_iuf_test20, pearson_weight_iuf_test20, pearson_r_a_test20, pearson_r_u_test20, data_arr, 401, 30)

In [22]:
write_file(pearson_iuf_out5, 'pearson_iuf_rating_res5_05282124.txt')
write_file(pearson_iuf_out10, 'pearson_iuf_rating_res10_05282124.txt')
write_file(pearson_iuf_out20, 'pearson_iuf_rating_res20_05282124.txt')

# Pearson Correlation With Case Amplification

In [23]:
# Calculate the case amplified weight matrix
def get_amp_pearson_weight_mat(weight_mat):
    for i in range(len(weight_mat)):
        for j in range(len(weight_mat[0])):
            weight_mat[i][j] *= math.pow(abs(weight_mat[i][j]),1.5)
    return weight_mat

In [24]:
pearson_amp_weight_test5 = get_amp_pearson_weight_mat(pearson_weight_test5)
pearson_amp_weight_test10 = get_amp_pearson_weight_mat(pearson_weight_test10)
pearson_amp_weight_test20 = get_amp_pearson_weight_mat(pearson_weight_test20)
pearson_amp_ranking_test5 = rank_pearson_weight(pearson_amp_weight_test5)
pearson_amp_ranking_test10 = rank_pearson_weight(pearson_amp_weight_test10)
pearson_amp_ranking_test20 = rank_pearson_weight(pearson_amp_weight_test20)

In [25]:
pearson_amp_out5 = pearson_rating_prediction(test5, pearson_amp_ranking_test5, pearson_amp_weight_test5, pearson_r_a_test5, pearson_r_u_test5, data_arr, 201, 30)
pearson_amp_out10 = pearson_rating_prediction(test10, pearson_amp_ranking_test10, pearson_amp_weight_test10, pearson_r_a_test10, pearson_r_u_test10, data_arr, 301, 30)
pearson_amp_out20 = pearson_rating_prediction(test20, pearson_amp_ranking_test20, pearson_amp_weight_test20, pearson_r_a_test20, pearson_r_u_test20, data_arr, 401, 30)

In [26]:
write_file(pearson_amp_out5, 'pearson_amp_rating_res5_05271750.txt')
write_file(pearson_amp_out10, 'pearson_amp_rating_res10_05271750.txt')
write_file(pearson_amp_out20, 'pearson_amp_rating_res20_05271750.txt')

# Item-Based Collaborative Filtering

In [27]:
def get_test_rated_movies_mat(test_data, movie_cnt, shift_id):
    res_arr = [[] for _ in range(100)]
    for i in test_data:
        user_id, movie_id, rating = list(i)[0] - shift_id, list(i)[1] - 1, list(i)[2]
        if rating != 0:
            res_arr[user_id].append((movie_id, rating))
        else:
            continue
    return res_arr

def get_adj_cos_sim(vec1, vec2, r_u):
    v1, v2 = [], []
    for i in range(len(vec1)):
        if vec1[i] != 0 and vec2[i] != 0:
            v1.append(vec1[i])
            v2.append(vec2[i])
    if len(v1) == 0: 
        return 0
    if len(v1) == 1:
        return 1 - abs(v1[0]-v2[0])/4  #weight 0, 0.25, 0.5, 0.75, 1 when only one similar item
    r_a, r_u = sum(v1)/len(v1), sum(v2)/len(v2)
    num, deno = 0.0, 0.0
    for i in range(len(v1)):
        v1[i] -= r_u 
        v2[i] -= r_u
    num = sum([m*n for m,n in zip(v1,v2)])
    deno = (sum([m*m for m in v1])**0.5) * (sum([n*n for n in v2])**0.5)
    if deno == 0: 
        return 0
    return num/deno

In [28]:
test5_rated_movie_mat = get_test_rated_movies_mat(test5, 5, 201)
test10_rated_movie_mat = get_test_rated_movies_mat(test10, 10, 301)
test20_rated_movie_mat = get_test_rated_movies_mat(test20, 20, 401)
test5_mean = pearson_r_a_test5
test10_mean = pearson_r_a_test10
test20_mean = pearson_r_a_test20
data_arr_t = numpy.array(data_arr).transpose()

In [29]:
def item_based_rating_prediction(test_data, train_data_t, test_movie_mat, test_mean, shift_id, k):
    res_arr = []
    for i in test_data:
        if list(i)[2] == 0:
            res_arr.append(list(i))
    for i,j in enumerate(res_arr):
        user_id = j[0] - shift_id
        movie_id = j[1] - 1
        cnt = 0
        num, deno = 0.0, 0.0
        sim_list = []
        for m in test_movie_mat[user_id]:
            sim_list.append(get_adj_cos_sim(train_data_t[m[0]], train_data_t[movie_id], test_mean[user_id]))
        rank_index = list(numpy.argsort(sim_list)[-k:])[::-1]
        for idx in rank_index:
            rating = test_movie_mat[user_id][idx][1]
            if sim_list[idx] > 0.3:
                num += rating*sim_list[idx]
                deno += sim_list[idx]
        if num == 0 and deno == 0: # rate 3 for default when there are no similar user
            res_arr[i][2] = 3
            continue
        ans = int(round(num/deno, 0))
        if ans > 5: ans = 5
        elif ans < 1: ans = 1
        res_arr[i][2] = ans
    return res_arr

In [30]:
item_out5 = item_based_rating_prediction(test5, data_arr_t, test5_rated_movie_mat, test5_mean, 201, 4)
item_out10 = item_based_rating_prediction(test10, data_arr_t, test10_rated_movie_mat, test10_mean, 301, 8)
item_out20 = item_based_rating_prediction(test20, data_arr_t, test20_rated_movie_mat, test20_mean, 401, 15)

In [31]:
write_file(item_out5, 'item_base_res5_05281927.txt')
write_file(item_out10, 'item_base_res10_05281927.txt')
write_file(item_out20, 'item_base_res20_05281927.txt')

# Own Algorithm

In [32]:
#Own algorithm get rating from weighted Cosine Similarity and Pearson Correlation
def own_alg(out1, out2):
    res_arr = []
    for i,j in zip(out1, out2):
        #mean = int(round((i[2]+j[2])/2, 0))
        mean = int(round(i[2]*0.6+j[2]*0.4, 0))
        #mean = int(round(i[2]*0.7+j[2]*0.3, 0))
        if mean > 5: mean = 5
        if mean < 1: mean = 1
        res_arr.append([i[0], i[1], mean])
    return res_arr

In [33]:
own_out5 = own_alg(rate_res5, pearson_out5)
own_out10 = own_alg(rate_res10, pearson_out10)
own_out20 = own_alg(rate_res20, pearson_out20)

In [34]:
write_file(own_out5, 'own_rating_res5_05290014.txt')
write_file(own_out10, 'own_rating_res10_05290014.txt')
write_file(own_out20, 'own_rating_res20_05290014.txt')